In [1]:
# copy x train five times for each model
#different workflow patterns, (pca vs no pca, hyperopt vs no, high dim vs low)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from plotter import Plotter

### Data preperation

#### Load the data and generate descriptors

In [3]:
pest_data = pd.read_csv("Data/BinaryPesticideData.csv", delimiter=",")
pest_data[0:5]

,Pesticide Name,LOD,EPA Tolerance (ppm),Smiles,ExactMolWt,HeavyAtomMolWt,NumRadicalElectrons,NumValenceElectrons,HeavyAtomCount,NHOHCount,...,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D,Class
0,Procymidone,0.014,NT,CC12CC1(C)C(=O)N(C2=O)c3cc(Cl)cc(Cl)c3,283.016684,273.054,0,94,18,0,...,2.0,0.0,0.0,0.0,2.0,0.0,0.6,1.4,3,1
1,Fenamiphos,0.004,NT,CCO[P](=O)(NC(C)C)Oc1ccc(SC)c(C)c1,303.105801,281.188,0,108,19,1,...,1.0,0.0,0.0,0.0,1.0,2.0,0.8,7.0,10,1
2,Metribuzin,0.030,NT,CSC1=NN=C(C(=O)N1N)C(C)(C)C,214.088832,200.182,0,78,14,2,...,2.0,1.0,0.0,0.0,1.0,2.0,0.6,2.0,3,1
3,Fenamiphos sulfone,0.008,NT,CCO[P](=O)(NC(C)C)Oc1ccc(c(C)c1)[S](C)(=O)=O,335.095630,313.186,0,120,21,1,...,3.0,0.0,0.0,0.0,1.0,1.0,0.8,7.0,10,1
4,Methiocarb,0.043,NT,CNC(=O)Oc1cc(C)c(SC)c(C)c1,225.082350,210.193,0,82,15,1,...,1.0,1.0,0.0,0.0,1.0,1.0,0.6,4.0,10,1


In [4]:
pest_data.tail()

,Pesticide Name,LOD,EPA Tolerance (ppm),Smiles,ExactMolWt,HeavyAtomMolWt,NumRadicalElectrons,NumValenceElectrons,HeavyAtomCount,NHOHCount,...,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D,Class
235,Oxamyl oxime,0.001,T,CSC(=N/O)/C(=O)N(C)C,162.046299,152.134,0,58,10,1,...,2.0,0.0,1.0,0.0,0.0,1.0,0.6,3.0,10,2
236,Pyriproxyfen,0.014,T,CC(COc1ccc(Oc2ccccc2)cc1)Oc3ccccn3,321.136493,302.224,0,122,24,0,...,4.0,0.0,0.0,0.0,3.0,0.0,1.0,7.0,10,2
237,Boscalid,0.001,T,Clc1ccc(cc1)c2ccccc2NC(=O)c3cccnc3Cl,342.032668,331.117,0,114,23,1,...,2.0,1.0,0.0,0.0,3.0,0.0,0.6,4.0,10,2
238,Fenpropathrin,0.026,T,CC1(C)C(C(=O)OC(C#N)c2cccc(Oc3ccccc3)c2)C1(C)C,349.167794,326.246,0,134,26,0,...,3.0,0.0,0.0,0.0,2.0,2.0,1.0,6.6,10,2
239,Carfentrazone ethyl,0.003,T,CCOC(=O)C(Cl)Cc1cc(N2N=C(C)N(C(F)F)C2=O)c(F)cc1Cl,411.036431,398.083,0,142,26,0,...,2.0,0.0,0.0,0.0,2.0,0.0,0.8,7.0,10,2


In [5]:
pest_data.count()

Pesticide Name              240
LOD                         240
EPA Tolerance (ppm)         240
Smiles                      240
ExactMolWt                  240
                           ... 
FeatureHydrophobeCount3D    240
ConformerModelRMSD3D        240
EffectiveRotorCount3D       240
ConformerCount3D            240
Class                       240
Length: 61, dtype: int64

In [6]:
pest_data["Class"].dtype

dtype('int64')

In [7]:
pest_data.columns

Index(['Pesticide Name', 'LOD', 'EPA Tolerance (ppm)', 'Smiles', 'ExactMolWt',
       'HeavyAtomMolWt', 'NumRadicalElectrons', 'NumValenceElectrons',
       'HeavyAtomCount', 'NHOHCount', 'NOCount', 'NumAliphaticCarbocycles',
       'NumAliphaticHeterocycles', 'NumAliphaticRings',
       'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings',
       'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds',
       'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles',
       'NumSaturatedRings', 'RingCount', 'Halogens', 'MolecularFormula',
       'MolecularWeight', 'CanonicalSMILES', 'XLogP', 'ExactMass', 'TPSA',
       'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount',
       'RotatableBondCount', 'HeavyAtomCount.1', 'IsotopeAtomCount',
       'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount',
       'BondStereoCount', 'DefinedBondStereoCount', 'UndefinedBondStereoCount',
       'CovalentUnitCount', 'Volume3D', 'XStericQu

In [8]:
pest_data.describe()

,LOD,ExactMolWt,HeavyAtomMolWt,NumRadicalElectrons,NumValenceElectrons,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,...,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D,Class
count,240.000000,240.000000,240.000000,240.0,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,...,240.000000,240.00000,240.000000,240.000000,240.000000,240.000000,240.000000,240.00000,240.000000,240.000000
mean,0.013313,296.822371,282.050196,0.0,103.650000,19.129167,0.654167,4.279167,0.166667,0.183333,...,1.866667,0.53750,0.100000,0.441667,1.562500,0.712500,0.741667,5.40000,8.604167,1.708333
std,0.026594,81.867931,79.940445,0.0,29.072309,5.705828,0.892310,1.641889,0.445025,0.398393,...,1.267114,0.73656,0.374948,0.735198,0.974706,0.908266,0.206011,2.57331,2.774206,0.455480
min,0.000100,44.026215,40.021000,0.0,18.000000,3.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.400000,0.00000,1.000000,1.000000
25%,0.002000,237.882259,222.210000,0.0,82.000000,15.000000,0.000000,3.000000,0.000000,0.000000,...,1.000000,0.00000,0.000000,0.000000,1.000000,0.000000,0.600000,3.95000,10.000000,1.000000
50%,0.005000,299.589972,282.830500,0.0,104.000000,19.000000,0.000000,4.000000,0.000000,0.000000,...,2.000000,0.00000,0.000000,0.000000,2.000000,0.000000,0.800000,5.90000,10.000000,2.000000
75%,0.015000,346.802600,327.363750,0.0,122.500000,23.000000,1.000000,5.000000,0.000000,0.000000,...,3.000000,1.00000,0.000000,1.000000,2.000000,1.000000,0.800000,7.00000,10.000000,2.000000
max,0.250000,682.023324,660.220000,0.0,210.000000,38.000000,5.000000,10.000000,2.000000,2.000000,...,6.000000,4.00000,2.000000,3.000000,4.000000,6.000000,1.400000,13.00000,10.000000,2.000000


In [9]:
#pest_data.hist(column='Na_to_K', bins=100)

In [10]:
#pest_data.hist(column='Age', bins=50)

In [11]:
X = pest_data[['LOD', 'ExactMolWt', 'HeavyAtomMolWt', 'NumValenceElectrons', 'HeavyAtomCount', 'NHOHCount', 
                        'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 
                        'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 
                        'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 
                        'NumSaturatedHeterocycles', 'NumSaturatedRings', 'RingCount', 'Halogens', "MolecularWeight", 
                        "XLogP", "ExactMass", "TPSA", "Complexity", "Charge", "HBondDonorCount", 
                        "HBondAcceptorCount", "RotatableBondCount", "HeavyAtomCount", "IsotopeAtomCount", "AtomStereoCount", 
                        "DefinedAtomStereoCount", "UndefinedAtomStereoCount", "BondStereoCount", "DefinedBondStereoCount", 
                        "UndefinedBondStereoCount", "CovalentUnitCount","Volume3D", "XStericQuadrupole3D", "YStericQuadrupole3D", 
                        "ZStericQuadrupole3D", "FeatureCount3D", "FeatureAcceptorCount3D", "FeatureDonorCount3D", "FeatureAnionCount3D", 
                        "FeatureCationCount3D", "FeatureRingCount3D", "FeatureHydrophobeCount3D", "ConformerModelRMSD3D", 
                        "EffectiveRotorCount3D", "ConformerCount3D"]].values


y = pest_data[['Class']].values
#y[0:5]

X[0:2]

array([[1.40000000e-02, 2.83016684e+02, 2.73054000e+02, 9.40000000e+01,
        1.80000000e+01, 0.00000000e+00, 3.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 2.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 2.00000000e+00, 0.00000000e+00, 5.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 2.00000000e+00,
        3.00000000e+00, 2.00000000e+00, 2.84130000e+02, 3.00000000e+00,
        2.83016684e+02, 3.74000000e+01, 4.05000000e+02, 0.00000000e+00,
        0.00000000e+00, 2.00000000e+00, 1.00000000e+00, 1.80000000e+01,
        0.00000000e+00, 2.00000000e+00, 0.00000000e+00, 2.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        2.03300000e+02, 7.89000000e+00, 2.66000000e+00, 8.90000000e-01,
        4.00000000e+00, 2.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 2.00000000e+00, 0.00000000e+00, 6.00000000e-01,
        1.40000000e+00, 3.00000000e+00],
       [4.00000000e-03,

In [12]:
y_array = pest_data[['Class']].to_numpy()
y_array[0:5]

array([[1],
       [1],
       [1],
       [1],
       [1]], dtype=int64)

In [13]:
pest_feats_array = X
y = y_array

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pest_feats_scaled = StandardScaler().fit_transform(pest_feats_array)  # Previously, a huge bias without scaling

# one feature had 99% explanation

pest_feats_pca = PCA(n_components=18).fit(pest_feats_scaled)
pest_feats_pca_values = pest_feats_pca.transform(pest_feats_scaled)
pest_feats_pca_values

array([[-0.38172713,  7.34651081, -1.45784533, ..., -0.02354543,
         0.79472852,  0.61704106],
       [ 0.25380124, -0.83135807,  1.87731163, ...,  0.52952181,
         0.49706464, -0.78630769],
       [-4.05825424, -1.43292535, -1.11888741, ...,  2.27281405,
         0.11460249, -0.91583163],
       ...,
       [ 1.2837196 ,  0.42218071, -2.88301013, ..., -0.23199105,
        -0.53712472,  0.92457317],
       [ 4.49176828,  2.86421085,  0.29362712, ...,  0.0375824 ,
        -1.1256568 , -0.07445442],
       [ 4.76059474, -1.47722347,  0.04087042, ...,  0.81132719,
        -0.46446512,  1.01131178]])

In [15]:
pest_feats_pca.explained_variance_ratio_

array([0.29927445, 0.12520607, 0.09989821, 0.08433809, 0.05745664,
       0.04384194, 0.03960146, 0.03180862, 0.03003908, 0.02732521,
       0.02085093, 0.02022818, 0.01626626, 0.01596335, 0.01488377,
       0.01159804, 0.01033055, 0.00822051])

In [16]:
pest_feats_pca.explained_variance_ratio_.sum()

0.9571313434933594

In [17]:
#pest_data['Class'] = pest_data['Class'].astype(int)
#y = np.asanyarray(pest_data['Class'])
#y[0:5]

X = pest_feats_pca_values[:]
X.shape

(240, 18)

#### train_test_split

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True) #random_state = 3
print ('X_train and y_train shape: ', X_train.shape, y_train.shape)
print ('X_test and y_test shape: ', X_test.shape, y_test.shape)

X_train and y_train shape:  (192, 18) (192, 1)
X_test and y_test shape:  (48, 18) (48, 1)


In [19]:
from sklearn.preprocessing import StandardScaler

#X_train = StandardScaler().fit_transform(X_train)
X_train[0:5]
#X_test = StandardScaler().fit_transform(X_test)
X_test[0:2]

array([[ 2.30132811,  9.00339601,  1.43715577,  5.69360538,  1.57479383,
         2.03796642,  0.02266985,  0.13027883,  0.71854656, -1.86403486,
        -0.02577041,  1.11668183, -0.89910588, -0.37484482, -2.51025499,
         0.93575065,  0.41164074, -0.3922313 ],
       [-3.91787465, -2.22109521, -1.58609748,  0.68319942,  1.66298701,
         2.8286358 , -0.57638625, -0.76124606, -0.32590149, -0.07288123,
         1.48606566, -0.48308397,  0.49369886,  0.29908065,  0.39621372,
        -0.30632629,  0.21996092,  0.48291396]])

In [20]:
#Just to be sure

if X_train.shape[0] == y_train.shape[0]:
    print("Train set dimensions match and is equal to", X_train.shape, 'vs', y_train.shape)
else:
    print("Train set dimensions do not match")

Train set dimensions match and is equal to (192, 18) vs (192, 1)


### KNN HyperOpt - Optimisation of model hyperparameters

In [ ]:
from hpsklearn import HyperoptEstimator, any_classifier, any_preprocessing, svc #any_sparse_classifier
from hyperopt import fmin, tpe
from hpsklearn import k_neighbors_classifier

# preprocessing = []

model_KNN = HyperoptEstimator(classifier=k_neighbors_classifier('myKNN'), algo=tpe.suggest, max_evals=300, trial_timeout=500)

# perform the search
model_KNN.fit(X_train, y_train.ravel())

In [25]:
accuracy_KNN = model_KNN.score(X_test, y_test)
print("Accuracy_KNN: %.3f" % accuracy_KNN)

# model details
print(model_KNN.best_model())

Accuracy_KNN: 0.667
{'learner': KNeighborsClassifier(algorithm='brute', leaf_size=27, metric='euclidean',
                     n_jobs=1, n_neighbors=11, p=4.0414781693317305,
                     weights='distance'), 'preprocs': (PCA(n_components=18, whiten=True),), 'ex_preprocs': ()}


In [26]:
from sklearn.metrics import classification_report
y_hat_KNN = model_KNN.predict(X_test)
print(classification_report(y_test, y_hat_KNN))

              precision    recall  f1-score   support

           1       0.40      0.13      0.20        15
           2       0.70      0.91      0.79        33

    accuracy                           0.67        48
   macro avg       0.55      0.52      0.49        48
weighted avg       0.60      0.67      0.61        48



#### Cross-validation for better accuracy

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

k = 4

model = KNeighborsClassifier(n_neighbors = k, weights="distance").fit(X_train, y_train.ravel()) #KNN_4

score = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy", n_jobs=-1).mean()
print(score)

0.5728744939271254
